In [1]:
import requests
from bs4 import BeautifulSoup

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [6]:
r = requests.get('https://www.morthoj.org/past-issues.php')

In [10]:
soup = BeautifulSoup(r.content)
a = soup.find_all('a')
pages = ['https://www.morthoj.org/' + a_.get('href') for a_ in a if 'index.php' in a_.get('href')]
pages

['https://www.morthoj.org//index.php',
 'https://www.morthoj.org//index.php',
 'https://www.morthoj.org/index.php',
 'https://www.morthoj.org/http://www.globalhealthlibrary.net/php/index.php',
 'https://www.morthoj.org/2023/v17n1/index.php',
 'https://www.morthoj.org/2022/v16n3/index.php',
 'https://www.morthoj.org/2022/v16n2/index.php',
 'https://www.morthoj.org/2022/v16n1/index.php',
 'https://www.morthoj.org/2021/v15n3/index.php',
 'https://www.morthoj.org/2021/v15n2/index.php',
 'https://www.morthoj.org/2021/v15n1/index.php',
 'https://www.morthoj.org/2020/v14n3/index.php',
 'https://www.morthoj.org/2020/v14n2/index.php',
 'https://www.morthoj.org/2020/v14n1/index.php',
 'https://www.morthoj.org/2019/v13n3/index.php',
 'https://www.morthoj.org/2019/v13n2/index.php',
 'https://www.morthoj.org/2019/v13n1/index.php',
 'https://www.morthoj.org/2018/v12n3/index.php',
 'https://www.morthoj.org/2018/v12n2/index.php',
 'https://www.morthoj.org/2018/v12n1/index.php',
 'https://www.morthoj.o

In [24]:
from tqdm import tqdm

urls = []
for p in tqdm(pages):
    r = requests.get(p)
    soup = BeautifulSoup(r.content)
    a = soup.find_all('a')
    a = [(p, a_.get('href')) for a_ in a if a_.get('href').endswith('.pdf')]
    urls.extend(a)
    
urls

100%|███████████████████████████████████████████| 63/63 [00:37<00:00,  1.68it/s]


[('https://www.morthoj.org//index.php', '2023/v17n2/contents.pdf'),
 ('https://www.morthoj.org//index.php',
  '2023/v17n2/fracture-liaison-service.pdf'),
 ('https://www.morthoj.org//index.php',
  '2023/v17n2/femoral-tunnel-placement.pdf'),
 ('https://www.morthoj.org//index.php', '2023/v17n2/AMR-sign-meniscus.pdf'),
 ('https://www.morthoj.org//index.php',
  '2023/v17n2/total-femur-replacement.pdf'),
 ('https://www.morthoj.org//index.php', '2023/v17n2/intramedullary-nail.pdf'),
 ('https://www.morthoj.org//index.php',
  '2023/v17n2/transtubular-transoral-approach.pdf'),
 ('https://www.morthoj.org//index.php',
  '2023/v17n2/capsulitis-COVID-vaccination.pdf'),
 ('https://www.morthoj.org//index.php',
  '2023/v17n2/open-tendoachilles-injury.pdf'),
 ('https://www.morthoj.org//index.php',
  '2023/v17n2/infrapatellar-branch-saphenous.pdf'),
 ('https://www.morthoj.org//index.php', '2023/v17n2/serum-procalcitonin.pdf'),
 ('https://www.morthoj.org//index.php', '2023/v17n2/pyomyositis.pdf'),
 ('http

In [31]:
urls_combined = []
for u in urls:
    u_ = u[0].split('/index.php')[0]
    urls_combined.append(f'{u_}/{u[1]}')

In [32]:
urls_combined = sorted(list(set(urls_combined)))
len(urls)

3806

In [33]:
!mkdir morthoj.org

mkdir: cannot create directory ‘morthoj.org’: File exists


In [34]:
urls_combined[:2]

['https://www.morthoj.org//2023/v17n2/AMR-sign-meniscus.pdf',
 'https://www.morthoj.org//2023/v17n2/bone-health-screening.pdf']

In [35]:
import os

pdfs = urls_combined
for i in tqdm(range(len(pdfs))):
    filename = os.path.join('morthoj.org', f'{i}.pdf')
    if os.path.exists(filename):
        continue
        
    u = pdfs[i]
        
    r = requests.get(u)
    with open(filename, 'wb') as fopen:
        fopen.write(r.content)

100%|███████████████████████████████████████| 3784/3784 [50:21<00:00,  1.25it/s]


In [36]:
from glob import glob
from tika import parser
import json

TIKA_HOST = 'http://localhost:9998'

In [37]:
files = glob('morthoj.org/*.pdf')

with open('morthoj.org.jsonl', 'w') as fopen:
    for f in tqdm(files):
        try:
            raw_xml = parser.from_file(f, TIKA_HOST, xmlContent=True)
            body = BeautifulSoup(raw_xml['content']).find('body')
            t = '\n'.join(body.findAll(text=True))
            fopen.write(f'{json.dumps(t)}\n')
        except:
            pass

100%|███████████████████████████████████████| 3784/3784 [00:55<00:00, 68.05it/s]
